# Keras Deep Learning Example Data Generation

In [8]:
from keras.models import Model, model_from_json
from keras import layers
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Activation, Flatten, Dense, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate
from keras.optimizers import Adam, SGD
from keras.metrics import binary_crossentropy, mean_absolute_error, categorical_crossentropy, mean_squared_error, hinge
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, CSVLogger, BaseLogger, Callback, EarlyStopping, TensorBoard, LambdaCallback, ReduceLROnPlateau
from keras import backend as K
from keras.utils.np_utils import to_categorical

from sklearn.metrics import precision_score, recall_score, auc, confusion_matrix, roc_curve, fbeta_score

Using TensorFlow backend.


In [ ]:
def get_BiometryNet():
    
    ## Conv Model
    map_input = Input((im_w, im_h, 2))
    
    filters_init = 16
    kernel_size = (3,3)
    batch_norm = True
    dropout = False
    
    block1 = conv_conv_MaxPool_block(map_input, filters_init, kernel_size, batch_norm, dropout, 2)
    
    block2 = conv_conv_MaxPool_block(block1, filters_init*2, kernel_size, batch_norm, dropout, 2)
    
    block3 = conv_conv_MaxPool_block(block2, filters_init*4, kernel_size, batch_norm, dropout, 2)
    
    block4 = conv_conv_MaxPool_block(block3, filters_init*8, kernel_size, batch_norm, dropout, 2)
    
    flatten = Flatten()(block4)
    
    model_convA = Model(inputs=map_input, outputs=[flatten])
    model_convB = Model(inputs=map_input, outputs=[flatten])
    ## FC Model
    map_inputA = Input((im_w, im_h, 2))
    map_inputB = Input((im_w, im_h, 2))
    
    flattenA = model_convA(map_inputA)
    flattenB = model_convB(map_inputB)
    
    dense1 = concatenate([flattenA, flattenB], axis = 1)
    
    dense1 = Dense(2048, activation="relu")(dense1)
    
    dense2 = Dense(2048, activation="relu")(dense1)

    output = Dense(2, activation='softmax', kernel_initializer="glorot_normal")(dense2)
    
    model = Model(inputs=[map_inputA, map_inputB], outputs=[output])
    

    model.compile(optimizer=Adam(lr=0.00001), loss=categorical_crossentropy, metrics=['categorical_accuracy'])
    
    print('Biometry-net_5 Initialized')

    return model, model_convA

In [ ]:
from keras.utils import plot_model
#model, model_conv = get_BiometryNet()
path_results = '/home/rmoreta/Projects/Biometria/Results/general/'
plot_model(model, to_file=path_results+'model_BiometryNet5.png', show_shapes=True, show_layer_names=True)
plot_model(model_conv, to_file=path_results+'model_BiometryNet5_model_conv.png', show_shapes=True, show_layer_names=True)

In [ ]:
model, model_conv = get_BiometryNet()

** VGG **

In [4]:
def conv_block(x, filters, kernel_size, bn, num_conv, block):
    """
    # Arguments
    
    # Returns
    
    """    
    conv_name = 'block' + str(block) + '_conv'
    bn_name = 'block' + str(block) + '_bn'
    pool_name = 'block' + str(block) + '_pool'
    activ_name = 'block' + str(block) + '_activation'
    
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    
    
    for conv_i in range(num_conv):
        x = Conv2D(filters, kernel_size, name=conv_name + str(conv_i+1))(x)
        x = BatchNormalization(axis=bn_axis, name=bn_name + str(conv_i+1))(x) if bn else x
        x = Activation('relu', name=activ_name + str(conv_i+1))(x)
        
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name=pool_name)(x)
    return x

In [6]:
def vgg16(input_shape, num_classes):
    """
    # Arguments
    
    # Returns
    
    """
    kernel_size = (3,3)
    filters_init = 32
    bn = 1
    
    inputs = Input(shape=input_shape, name='input')
    
    block1 = conv_block(inputs, filters_init, kernel_size, bn, num_conv=2, block=1)
    
    block2 = conv_block(block1, filters_init*2, kernel_size, bn, num_conv=2, block=2)
    
    block3 = conv_block(block2, filters_init*4, kernel_size, bn, num_conv=3, block=3)
    
    block4 = conv_block(block3, filters_init*8, kernel_size, bn, num_conv=3, block=4)
    
    block5 = conv_block(block4, filters_init*16, kernel_size, bn, num_conv=3, block=5)
    
    flatten = Flatten(name='flatten')(block5)
    
    fc1 = Dense(2048, activation='relu', name='fc1')(flatten)
    
    fc2 = Dense(2048, activation='relu', name='fc2')(fc1)
    
    predictions = Dense(num_classes, activation='softmax', name='predictions')(fc2)
    
    model = Model(inputs=[inputs], outputs=[predictions], name='vgg16')
    
    return model

In [9]:
model = vgg16((256,256,2), 2)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 256, 256, 2)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 254, 254, 32)      608       
_________________________________________________________________
block1_bn1 (BatchNormalizati (None, 254, 254, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 254, 254, 32)      0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 252, 252, 32)      9248      
_________________________________________________________________
block1_bn2 (BatchNormalizati (None, 252, 252, 32)      128       
_________________________________________________________________
activation_2 (Activation)    (None, 252, 252, 32)      0         
__________

** VGG16-Biometry **

In [ ]:
def conv_block(x, filters, kernel_size, bn, num_conv, block):
    """
    # Arguments
    
    # Returns
    
    """    
    conv_name = 'block' + str(block) + '_conv'
    bn_name = 'block' + str(block) + '_bn'
    pool_name = 'block' + str(block) + '_pool'
    activ_name = 'block' + str(block) + '_activation'
    
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    
    
    for conv_i in range(num_conv):
        x = Conv2D(filters, kernel_size, name=conv_name + str(conv_i+1))(x)
        x = BatchNormalization(axis=bn_axis, name=bn_name + str(conv_i+1))(x) if bn else x
        x = Activation('relu', name=activ_name + str(conv_i+1))(x)
        
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name=pool_name)(x)
    return x

In [ ]:
def vgg16_biometry(input_shape, num_classes, option):
    """
    # Arguments
    
    # Returns
    
    """
    kernel_size = (3,3)
    filters_init = 32
    bn = 1
    
    inputs = Input(shape=input_shape, name='inputs_conv')
    
    block1 = conv_block(inputs, filters_init, kernel_size, bn, num_conv=2, block=1)
    
    block2 = conv_block(block1, filters_init*2, kernel_size, bn, num_conv=2, block=2)
    
    block3 = conv_block(block2, filters_init*4, kernel_size, bn, num_conv=3, block=3)
    
    block4 = conv_block(block3, filters_init*8, kernel_size, bn, num_conv=3, block=4)
    
    block5 = conv_block(block4, filters_init*16, kernel_size, bn, num_conv=3, block=5)
    
    flatten = Flatten(name='flatten')(block5)
    
    model_conv = Model(inputs=inputs, outputs=[flatten])
    
    
    map_inputA = Input((im_w, im_h, 2))
    map_inputB = Input((im_w, im_h, 2))
    
    
    flattenA = model_conv(map_inputA)
    if option == 1: flattenB = model_conv(map_inputB)
    elif option == 2: flattenB = model_convB(map_inputB)
    
    conc = concatenate([flattenA, flattenB], axis = 1)
    
    fc1 = Dense(2048, activation='relu', name='fc1')(conc)
    
    fc2 = Dense(2048, activation='relu', name='fc2')(fc1)
    
    predictions = Dense(num_classes, activation='softmax', name='predictions')(fc2)
    
    model = Model(inputs=[map_inputA, map_inputB], outputs=[predictions], name='vgg16_biometry')
    
    return model

**AlexNet**

** GoogLeNet **

** U-Net **

In [ ]:
def get_unet():
    
    inputs = Input((512, 512, 1))
    
    conv1 = Conv2D(32, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(inputs)
    conv1 = Conv2D(32, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(64, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(pool1)
    conv2 = Conv2D(64, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(pool2)
    conv3 = Conv2D(128, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(pool3)
    conv4 = Conv2D(256, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    
    conv5 = Conv2D(512, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(pool4)
    conv5 = Conv2D(512, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(conv5)
    
    
    up6 = concatenate([Conv2D(256, (2, 2),  padding="same", kernel_initializer="glorot_normal", activation="relu")(UpSampling2D(size=(2, 2))(conv5)), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(up6)
    conv6 = Conv2D(256, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(conv6)

    up7 = concatenate([Conv2D(128, (2, 2),  padding="same", kernel_initializer="glorot_normal", activation="relu")(UpSampling2D(size=(2, 2))(conv6)), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(up7)
    conv7 = Conv2D(128, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(conv7)

    up8 = concatenate([Conv2D(64,(2, 2),  padding="same", kernel_initializer="glorot_normal", activation="relu")(UpSampling2D(size=(2, 2))(conv7)), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(up8)
    conv8 = Conv2D(64, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(conv8)

    up9 = concatenate([Conv2D(32,(2, 2),  padding="same", kernel_initializer="glorot_normal", activation="relu")(UpSampling2D(size=(2, 2))(conv8)), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(up9)
    conv9 = Conv2D(32, (3, 3), padding="same", kernel_initializer="glorot_normal", activation="relu")(conv9)

    conv10 = Conv2D(num_classes, (1, 1), activation='sigmoid', kernel_initializer="glorot_normal")(conv9)
    
    
    model = Model(outputs=[conv10], inputs=inputs)

    model.compile(optimizer=Adam(lr=0.0001),loss=categorical_crossentropy, metrics=['accuracy'])
    
    print('U-net Initialized')

    return model

In [31]:
def unet_downblock(x, filters, kernel_size, bn, block):
    """
    # Arguments
    
    # Returns
    
    """
    conv_name = 'block' + str(block) + '_conv'
    bn_name = 'block' + str(block) + '_bn'
    pool_name = 'block' + str(block) + '_bn'
    activ_name = 'block' + str(block) + '_activation'
    
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    
    
    x = Conv2D(filters, kernel_size, padding="same", name=conv_name+'1')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name+'1')(x) if bn else x
    x = Activation('relu', name=activ_name+'1')(x)
    
    x = Conv2D(filters, kernel_size, padding="same", name=conv_name+'2')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name+'2')(x) if bn else x
    x = Activation('relu', name=activ_name+'2')(x)
    
    pool = MaxPooling2D(pool_size=(2, 2), name=pool_name)(x)
    
    return pool, x

In [39]:
def unet_upblock(x, conv_down, filters, kernel_size, bn, block):
    """
    # Arguments
    
    # Returns
    
    """
    conv_name = 'block' + str(block) + '_conv'
    bn_name = 'block' + str(block) + '_bn'
    pool_name = 'block' + str(block) + '_bn'
    activ_name = 'block' + str(block) + '_activation'
    
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
        
    
    up = UpSampling2D(size=(2, 2))(x)
    conv_up = Conv2D(filters, (2, 2),  padding="same", activation="relu", name=conv_name + 'Up')(up)
    concat = concatenate([conv_up, conv_down], axis=3, name='block'+str(block)+'_conc')
    
    x = Conv2D(filters, kernel_size, padding="same", name=conv_name+'1')(concat)
    x = BatchNormalization(axis=bn_axis, name=bn_name+'1')(x) if bn else x
    x = Activation('relu', name=activ_name+'1')(x)
    
    x = Conv2D(filters, kernel_size, padding="same", name=conv_name + '2')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name+'2')(x) if bn else x
    x = Activation('relu', name=activ_name+'2')(x)
    
    return x

In [40]:
def unet(input_shape, num_classes):
    """
    # Arguments
    
    # Returns
    
    """
    
    kernel_size = (3,3)
    filters_init = 32
    bn = 0
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    
    
    inputs = Input(shape=input_shape)
    
    poolBlock1, convBlock1 = unet_downblock(inputs, filters_init, kernel_size, bn, block=1)
    
    poolBlock2, convBlock2 = unet_downblock(poolBlock1, filters_init*2, kernel_size, bn, block=2)
    
    poolBlock3, convBlock3 = unet_downblock(poolBlock2, filters_init*4, kernel_size, bn, block=3)
    
    poolBlock4, convBlock4 = unet_downblock(poolBlock3, filters_init*8, kernel_size, bn, block=4)
    
    
    convBlock5 = Conv2D(filters_init*16, (3, 3), padding="same", name='block5_conv1')(poolBlock4)
    convBlock5 = BatchNormalization(axis=bn_axis, name='block5_bn1')(convBlock5) if bn else convBlock5
    convBlock5 = Activation('relu', name='block5_activation1')(convBlock5)
    convBlock5 = Conv2D(filters_init*16, (3, 3), padding="same", name='block5_conv2')(convBlock5)
    convBlock5 = BatchNormalization(axis=bn_axis, name='block5_bn2')(convBlock5) if bn else convBlock5
    convBlock5 = Activation('relu', name='block5_activation2')(convBlock5)
    
    
    convBlock6 = unet_upblock(convBlock5, convBlock4, filters_init*8, kernel_size, bn, block=6)
    
    convBlock7 = unet_upblock(convBlock6, convBlock3, filters_init*4, kernel_size, bn, block=7)
    
    convBlock8 = unet_upblock(convBlock7, convBlock2, filters_init*2, kernel_size, bn, block=8)
    
    convBlock9 = unet_upblock(convBlock8, convBlock1, filters_init, kernel_size, bn, block=9)
    
    
    convBlock10 = Conv2D(num_classes, (1, 1), activation='sigmoid', name='block10_conv')(convBlock9)
    
    model = Model(outputs=[convBlock10], inputs=inputs, name='u-net')
    
    return model
    

In [41]:
model = unet((512,512,1), 2)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_13 (InputLayer)            (None, 512, 512, 1)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 512, 512, 32)  320         input_13[0][0]                   
____________________________________________________________________________________________________
block1_activation1 (Activation)  (None, 512, 512, 32)  0           block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv2 (Conv2D)            (None, 512, 512, 32)  9248        block1_activation1[0][0]         
___________________________________________________________________________________________

** ResNet **

In [ ]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    
    return x

In [ ]:
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    
    return x

In [ ]:
def ResNet50(input_shape, classes):

    img_input = Input(shape=input_shape)
            
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    x = Conv2D(
        64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = AveragePooling2D((3, 3), name='avg_pool')(x)


    x = Flatten()(x)
    x = Dense(classes, activation='softmax', name='fc')(x)

    inputs = img_input
    
    # Create model.
    model = Model(inputs, x, name='resnet50')
    
    print 'ResNet50 Initialized'
    return model

# Training

In [ ]:
model.compile

In [ ]:
class_weight = {0: , 1:}

In [ ]:
path_net = '/home/rmoreta/Projects/Biometria/Results/nets/'
name_net = 'Biometria_Prueba5_12epochs.hdf5'
model.load_weights(path_net + name_net)

In [ ]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.train_losses = []
        self.val_losses = []
        self.train_acc = []
        self.val_acc = []

    def on_batch_end(self, batch, logs={}):
        self.train_losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.train_acc.append(logs.get('categorical_accuracy'))
        self.val_acc.append(logs.get('val_categorical_accuracy'))
        
    def on_epoch_end(self, batch, logs={}):
        pass
history = LossHistory()

In [ ]:
# Save model in .hdf5
path_unet = '/home/rmoreta/Projects/PectoralisSegmentation/Results/'
name_unet = 'unet_nc' + str(num_classes) + '_pecs_' + str(num_images_train) + 'prueba_k2' + '.hdf5'
model_checkpoint = ModelCheckpoint(path_unet + name_unet, monitor='loss',verbose=1, save_best_only=True)

# Create a CVS with the values of loss and accuracy at each epoch
model_CVSLogger = CSVLogger('hola.cvs', separator=',', append=True)

# Gets loss values at each batch and epoch in a list
history = LossHistory()

# Early Stopping 
model_EarlyStopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

# Reduce LR
model_ReduceLROnPlateau = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=1, mode='auto', 
                  cooldown=0, min_lr=0.00005) # epsilon=0.0001

# Tensorboard
model_TensorBoard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False

## Fit, test, predict

In [ ]:
batch_size = 16
nb_epoch = 5


# train_mapasA = train_mapas[:,:,:,0].reshape([num_train_mapas,141,141,1])
# train_mapasB = train_mapas[:,:,:,1].reshape([num_train_mapas,141,141,1])

# model.fit(train_mapas, categorical_labels, batch_size=batch_size, epochs=nb_epoch, 
#           verbose=1, shuffle=True)

model.fit([train_mapas, train_mapas2], train_cat_labels, 
          validation_data = ([val_mapas, val_mapas2], val_cat_labels),
          batch_size=batch_size, epochs=nb_epoch, 
          verbose=1, shuffle=True,
         callbacks=[history])

In [ ]:
y_pred = model.predict()

In [ ]:
model.test()

## train_on_batch, test_on_batch, predict_on_batch

In [ ]:
num_epochs = 2

for epoch_i in range(num_epochs):
    batch_size = 
    num_batches = num_train_sampels // batch_size
    
    for batch_i in range(num_batches):
        inputA =
        inputB = 
        cat_labels = 
        
        model.train_on_batch([inputA, inputB], cat_labels)
        
        model.test_on_batch([test_mapas, test_mapas2], test_cat_labels)

In [ ]:
num_epochs = 2
# batch_size = 16
# seed = 9
loss = list()
loss_epoch = list()

for epoch_i in range(num_epochs):
    print 'Epoch', epoch_i+1
    num_batches = len(train_index_list_random_batch)
    
    for batch_i in range(num_batches):
        print 'Batch:', batch_i, '/', num_batches
        inputA, train_labels, n = data_preparation(curv_back_OD, subject_matrix, registros_matrix, train_index_list_random_batch[batch_i])
        inputB, train_labels2, n = data_preparation(curv_front_OD, subject_matrix, registros_matrix, train_index_list_random_batch[batch_i])
        cat_labels =  to_categorical(train_labels, num_classes=2)
        
        aux_loss = model.train_on_batch([inputA, inputB], cat_labels)
        
        loss.append(aux_loss)
        
        print 'Batch:', batch_i, '/', num_batches, ', loss:', aux_loss
    loss_epoch.append(loss)

In [ ]:
test_loss = list()
test_acc = list()
num_batches = len(test_index_list_random_batch)
for batch_i in range(num_batches):
    print 'Batch:', batch_i+1, '/', num_batches
    inputA, test_labels, n = data_preparation(curv_back_OD, subject_matrix, registros_matrix, test_index_list_random_batch[batch_i])
    inputB, test_labels2, n = data_preparation(curv_front_OD, subject_matrix, registros_matrix, test_index_list_random_batch[batch_i])
    test_cat_labels =  to_categorical(test_labels, num_classes=2)
    
    aux_loss, aux_acc = model.test_on_batch([inputA, inputB], test_cat_labels)
    
    test_loss.append(aux_loss)
    test_acc.append(aux_acc)

## fit_generator, evaluate_generator, predict_generator

In [ ]:
def get_random_subj_indexes_from_matrix(subject_matrix, subj_list, seed, train):
    if train == 1:
        aux_index = get_train_index(subject_matrix, subj_list)
    else:
        aux_index = get_test_index(subject_matrix, subj_list)

    np.random.seed(seed)
    aux_index_pos = np.where(aux_index)[0]
    np.random.shuffle(aux_index_pos)
    
    return aux_index_pos

def get_batch_indexes_balanced_labels(labels_matrix, index_list, batch_size):

    data_len = len(index_list)

    num_labels_1 = sum(labels_matrix[index_list])
    num_labels_0 = data_len - num_labels_1
    print 'data_len', data_len
    print 'num_labels_1', num_labels_1
    
    num_batches = int(np.ceil(float(data_len)/batch_size))
    print 'num_batches:', num_batches

    aux_index_1 = index_list[np.where(labels_matrix[index_list] == 1)]
    aux_index_0 = index_list[np.where(labels_matrix[index_list] == 0)]

    num_1_per_batch = float(num_labels_1)/data_len*batch_size
#     num_0_per_batch = batch_size - num_1_per_batch

    numero = num_labels_1 - num_batches*int(num_1_per_batch)
    if numero<0: numero = 0
    print 'numero', numero
    print 'num_1_per_batch', num_1_per_batch

    index_list_batch = []
    aux_0 = 0
    aux_1 = 0
    for i in range(num_batches):

#         if i == 0 and numero == 0:
#             num_1_per_batch = int(np.ceil(num_1_per_batch)) - 1
#             num_0_per_batch = batch_size - num_1_per_batch + 1
            
        if i == 0:
            num_1_per_batch = int(np.ceil(num_1_per_batch))
            num_0_per_batch = batch_size - num_1_per_batch
            print num_1_per_batch

        if i == numero:
            num_1_per_batch -= 1
            num_0_per_batch += 1

        aux_index_batch = aux_index_0[aux_0:aux_0+num_0_per_batch]
        aux_index_batch = np.concatenate((aux_index_batch, aux_index_1[aux_1:aux_1+num_1_per_batch]))
        aux_0 += num_0_per_batch
        aux_1 += num_1_per_batch

        np.random.shuffle(aux_index_batch)

        index_list_batch.extend(aux_index_batch)
    
    index_list_batch = np.array(index_list_batch)
#     np.random.shuffle(index_list_batch)
    return index_list_batch


def data_preparation(mapas_dataset, subject_matrix, registros_matrix, index_list):
    data_mapas, data_labels = [], []
    im_w = mapas_dataset[0][0].shape[0]
    im_h = mapas_dataset[0][0].shape[1]
    
    for i in range(len(index_list)):
        subj_i = subject_matrix[index_list[i], 0]
        subj_j = subject_matrix[index_list[i], 1]

        reg_i = registros_matrix[index_list[i], 0]
        reg_j = registros_matrix[index_list[i], 1]

        if subj_i == subj_j:
            aux_label = 1
        else:
            aux_label = 0

        aux_mapas = np.zeros([im_w, im_h, 2])
        aux_mapas[:,:,0] = mapas_dataset[subj_i][reg_i]
        aux_mapas[:,:,1] = mapas_dataset[subj_j][reg_j]

        data_mapas.append(aux_mapas)
        data_labels.append(aux_label)

    data_mapas = np.array(data_mapas)
    data_labels = np.array(data_labels, dtype='int8')
    num_data_mapas = data_mapas.shape[0]

    return data_mapas, data_labels, num_data_mapas

In [ ]:
import numpy as np
from keras.utils.np_utils import to_categorical

class DataGenerator(object):
    'Generates data for Keras'
    def __init__(self, dim_x = 141, dim_y = 141, num_channels = 2, batch_size = 32):
        'Initialization'
        self.dim_x = dim_x
        self.dim_y = dim_y
        self.num_channels = num_channels
        self.batch_size = batch_size

    def generate(self, mapas_dataset, subject_matrix, registros_matrix, labels_matrix, subj_list, seed, param_train):
        'Generates batches of samples'
        # Infinite loop
        while 1:
            # Generate order of exploration of dataset  
            index_list_random = self.__get_random_subj_indexes_from_matrix(subject_matrix, subj_list, seed, param_train)
            index_list_random_batch = self.__get_batch_indexes_balanced_labels(labels_matrix, index_list_random, self.batch_size)

            # Generate batches
            num_batches = int(np.ceil(len(index_list_random)/float(self.batch_size)))
            for i in range(num_batches):
                # Find list of indexes for each batch
                index_list_temp = index_list_random_batch[i*self.batch_size:(i+1)*self.batch_size]

                # Generate data
                X, y = self.__data_generation(mapas_dataset, subject_matrix, registros_matrix, labels_matrix, index_list_temp)

                yield X, y


    def __get_random_subj_indexes_from_matrix(subject_matrix, subj_list, seed, param_train):
        if param_train == 1:
            aux_index = self.get_train_index(subject_matrix, subj_list)
        elif param_train == 0:
            aux_index = self.get_test_index(subject_matrix, subj_list)

        np.random.seed(seed)
        aux_index_pos = np.where(aux_index)[0]
        np.random.shuffle(aux_index_pos)

        return aux_index_pos
    
    def __get_batch_indexes_balanced_labels(labels_matrix, index_list, batch_size):

        data_len = len(index_list)

        num_labels_1 = sum(labels_matrix[index_list])
        num_labels_0 = data_len - num_labels_1

        num_batches = int(np.ceil(float(data_len)/batch_size))

        aux_index_1 = index_list[np.where(labels_matrix[index_list] == 1)]
        aux_index_0 = index_list[np.where(labels_matrix[index_list] == 0)]

        num_1_per_batch = float(num_labels_1)/data_len*batch_size

        numero = num_labels_1 - num_batches*int(num_1_per_batch)
        if numero<0: numero = 0

        index_list_batch = []
        aux_0 = 0
        aux_1 = 0
        
        for i in range(num_batches):

            if i == 0:
                num_1_per_batch = int(np.ceil(num_1_per_batch))
                num_0_per_batch = batch_size - num_1_per_batch
                print num_1_per_batch

            if i == numero:
                num_1_per_batch -= 1
                num_0_per_batch += 1

            aux_index_batch = aux_index_0[aux_0:aux_0+num_0_per_batch]
            aux_index_batch = np.concatenate((aux_index_batch, aux_index_1[aux_1:aux_1+num_1_per_batch]))
            aux_0 += num_0_per_batch
            aux_1 += num_1_per_batch

            np.random.shuffle(aux_index_batch)

            index_list_batch.extend(aux_index_batch)

        index_list_batch = np.array(index_list_batch)
        
        return index_list_batch

    def __data_generation(mapas_dataset, subject_matrix, registros_matrix, labels_matrix, index_list):
        'Generates data of batch_size samples' 
        # Initialization         
        data_mapas, data_labels = [], []

        # Get mapas
        for i in range(len(index_list)):
            subj_i = subject_matrix[index_list[i], 0]
            subj_j = subject_matrix[index_list[i], 1]

            reg_i = registros_matrix[index_list[i], 0]
            reg_j = registros_matrix[index_list[i], 1]

            aux_mapas = np.zeros([self.dim_x, self.dim_y, self.num_channels])
            aux_mapas[:,:,0] = mapas_dataset[subj_i][reg_i]
            aux_mapas[:,:,1] = mapas_dataset[subj_j][reg_j]

            aux_label = labels_matrix[index_list[i]]
            
            data_mapas.append(aux_mapas)
            data_labels.append(aux_label)

        data_mapas = np.array(data_mapas)
        data_labels = np.array(data_labels, dtype='int8')
        
        data_labels_cat = to_categorical(data_labels, num_classes=2)

        return data_mapas, data_labels_cat

    def get_test_index(data, test_subj_ix):
        test_index = False
        for subj_i in test_subj_ix:
            for subj_j in test_subj_ix:
                aux = (data[:,0] == subj_i) | (data[:,1] == subj_j)
                test_index |= aux

        return test_index

    def get_train_index(data, test_subj_ix):
        test_index = False
        for subj_i in test_subj_ix:
            for subj_j in test_subj_ix:
                aux = (data[:,0] == subj_i) & (data[:,1] == subj_j)
                test_index |= aux

        return test_index 

In [ ]:
import numpy as np

from keras.models import Sequential
from my_classes import DataGenerator

# Parameters
params = {'dim_x': 32,
          'dim_y': 32,
          'dim_z': 32,
          'batch_size': 32,
          'shuffle': True}

# Datasets
partition = # IDs
labels = # Labels

# Generators
training_generator = DataGenerator(**params).generate(labels, partition['train'])
validation_generator = DataGenerator(**params).generate(labels, partition['validation'])

# Design model
model = Sequential()
[...] # Architecture
model.compile()

# Train model on dataset
model.fit_generator(generator = training_generator,
                    steps_per_epoch = len(partition['train'])//batch_size,
                    validation_data = validation_generator,
                    validation_steps = len(partition['validation'])//batch_size)